In [8]:
/**
 *This code focuses on calibrating and debugging the existing open-source Kalman Filter from apache.commons-math. The current implementation exhibits two main limitations:

 Lack of native Kotlin support: Although the Java-based library can be used in Kotlin, it does not leverage Kotlin-specific features or idioms.

 Limited flexibility in system modeling: In the update step, the filter always assumes the same measurement covariance. In real-world applications, this covariance can change from one iteration to the next, reducing modeling accuracy when using a fixed value.
 * 
 */

In [1]:
USE {
    repositories {
        mavenCentral()
    }

    dependencies {
        implementation("org.apache.commons:commons-math3:3.6.1")
    }
}

In [2]:
%use kandy
%use dataframe

In [3]:
import kotlin.random.Random

// Prepare the measurement data 
fun getRandomMeasure(
    trueValue: Double,
    range: Double
): DoubleArray {

    val measureVal = Random.nextDouble(trueValue - range, trueValue + range)
    return  doubleArrayOf(measureVal)
}

In [4]:
import org.apache.commons.math3.filter.KalmanFilter
import org.apache.commons.math3.filter.DefaultProcessModel
import org.apache.commons.math3.filter.ProcessModel
import org.apache.commons.math3.filter.DefaultMeasurementModel
import org.apache.commons.math3.filter.MeasurementModel


val processModel: ProcessModel = DefaultProcessModel(
    arrayOf( //stateTransition: X
        doubleArrayOf(1.0)
    ),
    arrayOf( // control : U
        doubleArrayOf(1.0)
    ),
    arrayOf( // processNoise: Q
        doubleArrayOf(1.0)
    ),
    doubleArrayOf(50.0), // inital state estimate
    arrayOf( // initial covariance
        doubleArrayOf(100.0*100.0)
    ),
)

val measurementModel = DefaultMeasurementModel(
    arrayOf( //measMatrix: H
        doubleArrayOf(1.0)
    ),
    arrayOf( //measNoise : R
        doubleArrayOf(100.0*100)
    ),
)

val kalmanFilter = KalmanFilter(processModel, measurementModel)


In [5]:

val trueValue = 100.0
val range = 100.0


val estimate = mutableListOf<Double>()
val measurement = mutableListOf<Double>()
val covariances = mutableListOf<Double>()


repeat(100){ i ->
    
    kalmanFilter.predict()

    kalmanFilter.stateEstimation
    
    
    kalmanFilter.correct(
            getRandomMeasure(
                trueValue = trueValue ,
                range = range
            ).also {
                measurement.add(it[0])
            }
        )
    
    estimate.add(kalmanFilter.stateEstimation[0])
    covariances.add(kalmanFilter.errorCovariance[0][0])

}


val plotData = dataFrameOf(
    "data" to estimate +  measurement,
    "type" to  List(estimate.size) { "estimate" } + List(measurement.size) { "measurement" },
    "index" to List(estimate.size) { it } +  List(measurement.size) { it }
)


plotData.plot {
    line {
        x("index")
        y("data")
        color("type"){
            scale = categorical("measurement" to Color.BLUE, "estimate" to Color.RED )
        }
    }
    layout.size = 1000 to 450

}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="Y68YaK"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"data":[124.37648887862449,148.232024089952,112.85313002198401,98.13546691789809,89.26451091836702,102.40358703804088,102.30274283112121,107.13637858709708,100.96144993684916,92.74753785888488,93.66901258294249,91.00649674393412,90.59617986286652,85.46138351193187,81.22262070245938,79.81911322397085,76.54185554656699,73.13107004380913,79.07877880161662,83.28371869489322,79.7786319228032,81.52152588396204,78.79961666157024,81.78510611315816,80.16049429535275,77.5259202607246,76.22127700493718,78.44801678408595,78.73738502764792,76.91571114987465,75.55283661678173,76.57313434092076,76.9817371259102,77.07450655105863,76.98133557461293,76.99755856353869,75.9349009507047,78.92575372031595,79.22134188550217,82.1488211734351,80.20399377656493,79.17857875429858,80.75448264640235,79.95854942424671,78.54860783647177,79.6469241562097,78.10200079199667,77.90343791964425,79.33391383114663,78.84834051249263,80.7989239163075,81.55807455495172,83.297281655205,83.08272559912527,81.75750789455944,83.34568737887702,83.07536599705904,83.81780755184379,83.27665335001294,84.69406299145761,85.73811952560956,87.76987693824753,86.74774179300611,88.28634475804365,87.1711217671185,88.86353933756101,89.35044609945109,88.12860515411113,88.38105809680569,87.69904542525245,86.8799649380804,86.8470292209373,88.44969415954317,89.88834505092966,91.09602032603196,91.02957759756404,92.15437401768675,91.89957161031577,92.16577992843669,91.49197901436264,91.94266455193353,92.80899294131868,92.51114143452969,93.36391168492915,92.61572441410377,93.4744218397654,94.02115948722542,94.61689797400278,93.43032827015806,92.6382666337356,92.13622042532917,91.26688422001254,92.65645176141237,93.59396546848959,93.35617231561228,92.9443446735575,93.90834833478638,94.9144653984829,95.58496705890711,95.86699794369073,198.74554085205162,195.9311698454101,6.76595351703182,39.30893131537041,44.95846172788447,181.1186810762002,101.59824201814428,145.70714358718314,45.56244280775379,10.923261154156005,103.7588537298082,59.228293645208495,85.2954218437879,14.091065143344084,18.16177755049413,57.570694634700146,21.406381413933097,12.446545503765094,190.63818150163073,166.19565782072135,7.311268546207472,119.21689612899715,17.347216317797233,152.00826800529674,40.42004049042476,10.616056901294634,41.873481035021975,139.13307170090917,86.88983605401812,23.926106659273305,34.66911564884894,108.10274497508772,89.97567775229149,80.10747974126107,73.8526050634909,77.55662479326027,38.384776716411295,187.20807905219004,90.17778849461652,193.16538539911667,4.800447813621278,38.55801229984945,144.4990080802355,47.1036965768936,19.18948910013656,126.78165508027455,10.55265349113943,69.06256477696485,144.1602129387465,56.46125084847802,172.24930583106973,117.73571233471199,167.5093852329663,72.53154657927739,15.594317506403787,163.81595559565733,69.18012869550085,122.5209032727687,54.67735602014676,160.61093038087512,142.3930329837176,199.43625006890187,29.86687127493708,174.95579784094505,23.59987906309722,186.46301093251276,117.75000216502846,16.068593035364497,103.43205392063544,46.60521568139955,37.01365298061856,84.8214735321777,187.99453686316065,180.115146014576,167.55738692520373,86.78376224941515,164.68504010086272,75.32280104277338,109.63539752182044,46.897863966757484,122.01897555819767,151.09458930258887,72.31209350473387,151.64692707593272,41.08960900960339,153.0535740257313,132.23334055166842,136.55226267753332,9.318952776002476,36.10637941475674,56.062660027097124,28.391143520618066,193.8063960011647,162.2685774254734,75.82972826096817,62.40706182670544,165.81346797678424,170.396376

In [6]:
val convarianceDf = dataFrameOf(
    "covariances" to covariances,
    "index" to List(covariances.size) { it } 
)


convarianceDf.plot {
    line {
        x("index")
        y("covariances")
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="oDSwbZ"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"covariances":[5000.249987500624,3333.8888370420977,2500.8748687715583,2001.1997360659825,1668.1939811828634,1430.4278289267613,1252.1863850678317,1113.6280358163442,1002.8478076139929,912.2698034040341,836.843420767596,773.0720847674465,718.4582381454844,671.1703354422079,629.8347143222918,593.4009240802189,561.0519446448299,532.1427574779174,506.1573452039441,482.6779770605026,461.36287104472865,441.9296833264578,424.14312530865226,407.8055525938549,392.74972572811805,378.83317966898085,365.9337997964761,353.9463132326356,342.7794818975044,332.35383884502545,322.59984903631727,313.4564045181628,304.86958515857197,296.7916318279955,289.180090717308,281.9970964153263,275.20876818489006,268.7846991198072,262.6975219285071,256.92253826178563,251.43740099395492,246.22184083706924,241.25743023523174,236.52737873985254,232.01635507527212,227.71033191958392,223.5964500880168,219.6628993470773,215.89881353113978,212.29417799839769,208.83974776510266,205.52697490774605,202.34794403177756,199.29531478020547,196.3622705020883,193.5424723244345,190.83001797537003,188.2194047948589,185.7054964444241,183.28349289138865,180.9489032979278,178.69752149217197,176.52540373894266,174.4288485624656,172.40437840342162,170.44872291868447,168.55880375464102,166.73172064459595,164.96473869784833,163.25527676294757,161.6008967606908,159.99929389386875,158.44828765082127,156.9458135287095,155.4899154102119,154.0787385342384,152.71052300735005,151.3835978079703,150.09637524026562,148.84734579883184,147.63507340911468,146.45819101187308,145.31539646301218,144.2054487228117,143.12716431099156,142.07941400622312,141.06111977063864,140.07125188164053,139.10882625488432,138.17290194372762,137.2625788017162,136.37699529583304,135.51532645928202,134.67678197352194,133.860604370125,133.0660673438108,132.2924741687126,131.53915621057536,130.80547152816837,130.0908035577279],
"index":[0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0,30.0,31.0,32.0,33.0,34.0,35.0,36.0,37.0,38.0,39.0,40.0,41.0,42.0,43.0,44.0,45.0,46.0,47.0,48.0,49.0,50.0,51.0,52.0,53.0,54.0,55.0,56.0,57.0,58.0,59.0,60.0,61.0,62.0,63.0,64.0,65.0,66.0,67.0,68.0,69.0,70.0,71.0,72.0,73.0,74.0,75.0,76.0,77.0,78.0,79.0,80.0,81.0,82.0,83.0,84.0,85.0,86.0,87.0,88.0,89.0,90.0,91.0,92.0,93.0,94.0,95.0,96.0,97.0,98.0,99.0]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"index",
"y":"covariances"
},
"stat":"identity",
"sampling":"none",
"position":"identity",
"geom":"line",
"data":{
}
}]
};
 var plotContainer = document.getElementById("oDSwbZ");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M23.682003994035554 15.500000000000057 L23.682003994035554 15.500000000000057 L28.466247225153847 121.56880332480736 L33.25049045627214 174.59259996276447 L38.034733687390435 206.39839630088775 L42.818976918508724 227.59519503266762 L47.60322014962702 242.7297120813933 L52.38746338074532 254.07530583086367 L57.171706611863605 262.8949535355176 L61.9559498429819 269.9464418843903 L66.7401930741002 275.7119990755526 L71.52443630521849 280.51311085325136 L76.30867953633678 284.5723445278355 L81.09292276745508 288.04867698707585 L85.87716599857336 291.05869150289453 L90.66140922969166 293.6898257355131 L95.44565246080995 296.00894397137716 L100.22989569192825 298.06805194838824 L105.014138923046